# Word2Vec Tutorial

In case you missed the buzz, word2vec is a widely featured as a member of the “new wave” of machine learning algorithms based on neural networks, commonly referred to as "deep learning" (though word2vec itself is rather shallow). Using large amounts of unannotated plain text, word2vec learns relationships between words automatically. The output are vectors, one vector per word, with remarkable linear relationships that allow us to do things like vec(“king”) – vec(“man”) + vec(“woman”) =~ vec(“queen”), or vec(“Montreal Canadiens”) – vec(“Montreal”) + vec(“Toronto”) resembles the vector for “Toronto Maple Leafs”.

Word2vec is very useful in [automatic text tagging](https://github.com/RaRe-Technologies/movie-plots-by-genre), recommender systems and machine translation.

Check out an [online word2vec demo](http://radimrehurek.com/2014/02/word2vec-tutorial/#app) where you can try this vector algebra for yourself. That demo runs `word2vec` on the Google News dataset, of **about 100 billion words**.

## This tutorial

In this tutorial you will learn how to train and evaluate word2vec models on your business data.


## Preparing the Input
Starting from the beginning, gensim’s `word2vec` expects a sequence of sentences as its input. Each sentence is a list of words (utf8 strings):

In [1]:
# import modules & set up logging
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
sentences = [['first', 'sentence'], ['second', 'sentence']]
# train word2vec on the two sentences
model = gensim.models.Word2Vec(sentences, min_count=1)

2017-12-10 22:48:14,215 : INFO : collecting all words and their counts
2017-12-10 22:48:14,219 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-12-10 22:48:14,221 : INFO : collected 3 word types from a corpus of 4 raw words and 2 sentences
2017-12-10 22:48:14,223 : INFO : Loading a fresh vocabulary
2017-12-10 22:48:14,224 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2017-12-10 22:48:14,226 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2017-12-10 22:48:14,228 : INFO : deleting the raw counts dictionary of 3 items
2017-12-10 22:48:14,231 : INFO : sample=0.001 downsamples 3 most-common words
2017-12-10 22:48:14,233 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2017-12-10 22:48:14,236 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2017-12-10 22:48:14,262 : INFO : resetting layer weights
2017-12-10 22:48:14,272 : INFO : training model with 3 workers o

Keeping the input as a Python built-in list is convenient, but can use up a lot of RAM when the input is large.

Gensim only requires that the input must provide sentences sequentially, when iterated over. No need to keep everything in RAM: we can provide one sentence, process it, forget it, load another sentence…

For example, if our input is strewn across several files on disk, with one sentence per line, then instead of loading everything into an in-memory list, we can process the input file by file, line by line:

In [3]:
# create some toy data to use with the following example
import smart_open, os

if not os.path.exists('./data/'):
    os.makedirs('./data/')

filenames = ['./data/f1.txt', './data/f2.txt']

for i, fname in enumerate(filenames):
    with smart_open.smart_open(fname, 'w') as fout:
        for line in sentences[i]:
            fout.write(line + '\n')

In [4]:
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()

In [5]:
sentences = MySentences('./data/') # a memory-friendly iterator
print(list(sentences))

[['first'], ['sentence'], ['second'], ['sentence']]


In [6]:
# generate the Word2Vec model
model = gensim.models.Word2Vec(sentences, min_count=1)

2017-12-10 22:48:14,378 : INFO : collecting all words and their counts
2017-12-10 22:48:14,385 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-12-10 22:48:14,386 : INFO : collected 3 word types from a corpus of 4 raw words and 4 sentences
2017-12-10 22:48:14,389 : INFO : Loading a fresh vocabulary
2017-12-10 22:48:14,391 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2017-12-10 22:48:14,392 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2017-12-10 22:48:14,395 : INFO : deleting the raw counts dictionary of 3 items
2017-12-10 22:48:14,396 : INFO : sample=0.001 downsamples 3 most-common words
2017-12-10 22:48:14,398 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2017-12-10 22:48:14,400 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2017-12-10 22:48:14,401 : INFO : resetting layer weights
2017-12-10 22:48:14,403 : INFO : training model with 3 workers o

In [7]:
print(model)
print(model.wv.vocab)

Word2Vec(vocab=3, size=100, alpha=0.025)
{'second': <gensim.models.keyedvectors.Vocab object at 0x10c1ec610>, 'first': <gensim.models.keyedvectors.Vocab object at 0x10da66fd0>, 'sentence': <gensim.models.keyedvectors.Vocab object at 0x10da66650>}


Say we want to further preprocess the words from the files — convert to unicode, lowercase, remove numbers, extract named entities… All of this can be done inside the `MySentences` iterator and `word2vec` doesn’t need to know. All that is required is that the input yields one sentence (list of utf8 words) after another.

**Note to advanced users:** calling `Word2Vec(sentences, iter=1)` will run **two** passes over the sentences iterator. In general it runs `iter+1` passes. By the way, the default value is `iter=5` to comply with Google's word2vec in C language. 
  1. The first pass collects words and their frequencies to build an internal dictionary tree structure. 
  2. The second pass trains the neural model.

These two passes can also be initiated manually, in case your input stream is non-repeatable (you can only afford one pass), and you’re able to initialize the vocabulary some other way:

In [8]:
# build the same model, making the 2 steps explicit
new_model = gensim.models.Word2Vec(min_count=1)  # an empty model, no training
new_model.build_vocab(sentences)                 # can be a non-repeatable, 1-pass generator     
new_model.train(sentences, total_examples=new_model.corpus_count, epochs=new_model.iter)                       
# can be a non-repeatable, 1-pass generator

2017-12-10 22:48:14,443 : INFO : collecting all words and their counts
2017-12-10 22:48:14,445 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-12-10 22:48:14,448 : INFO : collected 3 word types from a corpus of 4 raw words and 4 sentences
2017-12-10 22:48:14,450 : INFO : Loading a fresh vocabulary
2017-12-10 22:48:14,452 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2017-12-10 22:48:14,453 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2017-12-10 22:48:14,455 : INFO : deleting the raw counts dictionary of 3 items
2017-12-10 22:48:14,457 : INFO : sample=0.001 downsamples 3 most-common words
2017-12-10 22:48:14,460 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2017-12-10 22:48:14,462 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2017-12-10 22:48:14,464 : INFO : resetting layer weights
2017-12-10 22:48:14,466 : INFO : training model with 3 workers o

0

In [9]:
print(new_model)
print(model.wv.vocab)

Word2Vec(vocab=3, size=100, alpha=0.025)
{'second': <gensim.models.keyedvectors.Vocab object at 0x10c1ec610>, 'first': <gensim.models.keyedvectors.Vocab object at 0x10da66fd0>, 'sentence': <gensim.models.keyedvectors.Vocab object at 0x10da66650>}


### More data would be nice
For the following examples, we'll use the [Lee Corpus](https://github.com/RaRe-Technologies/gensim/blob/develop/gensim/test/test_data/lee_background.cor) (which you already have if you've installed gensim):

In [10]:
# Set file names for train and test data
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data']) + os.sep
lee_train_file = test_data_dir + 'lee_background.cor'

In [11]:
class MyText(object):
    def __iter__(self):
        for line in open(lee_train_file):
            # assume there's one document per line, tokens separated by whitespace
            yield line.lower().split()

sentences = MyText()

print(sentences)

## Training
`Word2Vec` accepts several parameters that affect both training speed and quality.

### min_count
`min_count` is for pruning the internal dictionary. Words that appear only once or twice in a billion-word corpus are probably uninteresting typos and garbage. In addition, there’s not enough data to make any meaningful training on those words, so it’s best to ignore them:

In [12]:
# default value of min_count=5
model = gensim.models.Word2Vec(sentences, min_count=10)

2017-12-10 22:48:14,551 : INFO : collecting all words and their counts
2017-12-10 22:48:14,586 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-12-10 22:48:14,625 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2017-12-10 22:48:14,627 : INFO : Loading a fresh vocabulary
2017-12-10 22:48:14,646 : INFO : min_count=10 retains 806 unique words (7% of original 10186, drops 9380)
2017-12-10 22:48:14,649 : INFO : min_count=10 leaves 40964 word corpus (68% of original 59890, drops 18926)
2017-12-10 22:48:14,659 : INFO : deleting the raw counts dictionary of 10186 items
2017-12-10 22:48:14,661 : INFO : sample=0.001 downsamples 54 most-common words
2017-12-10 22:48:14,663 : INFO : downsampling leaves estimated 26224 word corpus (64.0% of prior 40964)
2017-12-10 22:48:14,665 : INFO : estimated required memory for 806 words and 100 dimensions: 1047800 bytes
2017-12-10 22:48:14,672 : INFO : resetting layer weights
2017-12-10 22:4

### size
`size` is the number of dimensions (N) of the N-dimensional space that gensim Word2Vec maps the words onto.

Bigger size values require more training data, but can lead to better (more accurate) models. Reasonable values are in the tens to hundreds.

In [13]:
# default value of size=100
model = gensim.models.Word2Vec(sentences, size=200)

2017-12-10 22:48:14,899 : INFO : collecting all words and their counts
2017-12-10 22:48:14,901 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-12-10 22:48:14,942 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2017-12-10 22:48:14,944 : INFO : Loading a fresh vocabulary
2017-12-10 22:48:14,968 : INFO : min_count=5 retains 1723 unique words (16% of original 10186, drops 8463)
2017-12-10 22:48:14,970 : INFO : min_count=5 leaves 46858 word corpus (78% of original 59890, drops 13032)
2017-12-10 22:48:14,984 : INFO : deleting the raw counts dictionary of 10186 items
2017-12-10 22:48:14,987 : INFO : sample=0.001 downsamples 49 most-common words
2017-12-10 22:48:14,989 : INFO : downsampling leaves estimated 32849 word corpus (70.1% of prior 46858)
2017-12-10 22:48:14,990 : INFO : estimated required memory for 1723 words and 200 dimensions: 3618300 bytes
2017-12-10 22:48:15,003 : INFO : resetting layer weights
2017-12-10 22:

### workers
`workers`, the last of the major parameters (full list [here](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)) is for training parallelization, to speed up training:

In [14]:
# default value of workers=3 (tutorial says 1...)
model = gensim.models.Word2Vec(sentences, workers=4)

2017-12-10 22:48:15,310 : INFO : collecting all words and their counts
2017-12-10 22:48:15,313 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-12-10 22:48:15,351 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2017-12-10 22:48:15,353 : INFO : Loading a fresh vocabulary
2017-12-10 22:48:15,368 : INFO : min_count=5 retains 1723 unique words (16% of original 10186, drops 8463)
2017-12-10 22:48:15,369 : INFO : min_count=5 leaves 46858 word corpus (78% of original 59890, drops 13032)
2017-12-10 22:48:15,382 : INFO : deleting the raw counts dictionary of 10186 items
2017-12-10 22:48:15,385 : INFO : sample=0.001 downsamples 49 most-common words
2017-12-10 22:48:15,386 : INFO : downsampling leaves estimated 32849 word corpus (70.1% of prior 46858)
2017-12-10 22:48:15,388 : INFO : estimated required memory for 1723 words and 100 dimensions: 2239900 bytes
2017-12-10 22:48:15,400 : INFO : resetting layer weights
2017-12-10 22:

The `workers` parameter only has an effect if you have [Cython](http://cython.org/) installed. Without Cython, you’ll only be able to use one core because of the [GIL](https://wiki.python.org/moin/GlobalInterpreterLock) (and `word2vec` training will be [miserably slow](http://rare-technologies.com/word2vec-in-python-part-two-optimizing/)).

## Memory
At its core, `word2vec` model parameters are stored as matrices (NumPy arrays). Each array is **#vocabulary** (controlled by min_count parameter) times **#size** (size parameter) of floats (single precision aka 4 bytes).

Three such matrices are held in RAM (work is underway to reduce that number to two, or even one). So if your input contains 100,000 unique words, and you asked for layer `size=200`, the model will require approx. `100,000*200*4*3 bytes = ~229MB`.

There’s a little extra memory needed for storing the vocabulary tree (100,000 words would take a few megabytes), but unless your words are extremely loooong strings, memory footprint will be dominated by the three matrices above.

## Evaluating
`Word2Vec` training is an unsupervised task, there’s no good way to objectively evaluate the result. Evaluation depends on your end application.

Google has released their testing set of about 20,000 syntactic and semantic test examples, following the “A is to B as C is to D” task. It is provided in the 'datasets' folder.

For example a syntactic analogy of comparative type is bad:worse;good:?. There are total of 9 types of syntactic comparisons in the dataset like plural nouns and nouns of opposite meaning.

The semantic questions contain five types of semantic analogies, such as capital cities (Paris:France;Tokyo:?) or family members (brother:sister;dad:?). 

Gensim supports the same evaluation set, in exactly the same format:

In [15]:
model.accuracy('./datasets/questions-words.txt')

2017-12-10 22:48:15,762 : INFO : precomputing L2-norms of word weight vectors
2017-12-10 22:48:15,811 : INFO : family: 0.0% (0/2)
2017-12-10 22:48:15,849 : INFO : gram3-comparative: 0.0% (0/12)
2017-12-10 22:48:15,866 : INFO : gram4-superlative: 0.0% (0/12)
2017-12-10 22:48:15,886 : INFO : gram5-present-participle: 0.0% (0/20)
2017-12-10 22:48:15,912 : INFO : gram6-nationality-adjective: 0.0% (0/20)
2017-12-10 22:48:15,936 : INFO : gram7-past-tense: 0.0% (0/20)
2017-12-10 22:48:15,955 : INFO : gram8-plural: 0.0% (0/12)
2017-12-10 22:48:15,967 : INFO : total: 0.0% (0/98)


[{'correct': [], 'incorrect': [], 'section': u'capital-common-countries'},
 {'correct': [], 'incorrect': [], 'section': u'capital-world'},
 {'correct': [], 'incorrect': [], 'section': u'currency'},
 {'correct': [], 'incorrect': [], 'section': u'city-in-state'},
 {'correct': [],
  'incorrect': [(u'HE', u'SHE', u'HIS', u'HER'),
   (u'HIS', u'HER', u'HE', u'SHE')],
  'section': u'family'},
 {'correct': [], 'incorrect': [], 'section': u'gram1-adjective-to-adverb'},
 {'correct': [], 'incorrect': [], 'section': u'gram2-opposite'},
 {'correct': [],
  'incorrect': [(u'GOOD', u'BETTER', u'GREAT', u'GREATER'),
   (u'GOOD', u'BETTER', u'LONG', u'LONGER'),
   (u'GOOD', u'BETTER', u'LOW', u'LOWER'),
   (u'GREAT', u'GREATER', u'LONG', u'LONGER'),
   (u'GREAT', u'GREATER', u'LOW', u'LOWER'),
   (u'GREAT', u'GREATER', u'GOOD', u'BETTER'),
   (u'LONG', u'LONGER', u'LOW', u'LOWER'),
   (u'LONG', u'LONGER', u'GOOD', u'BETTER'),
   (u'LONG', u'LONGER', u'GREAT', u'GREATER'),
   (u'LOW', u'LOWER', u'GOOD',

This `accuracy` takes an 
[optional parameter](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec.accuracy) `restrict_vocab` 
which limits which test examples are to be considered.



In the December 2016 release of Gensim we added a better way to evaluate semantic similarity.

By default it uses an academic dataset WS-353 but one can create a dataset specific to your business based on it. It contains word pairs together with human-assigned similarity judgments. It measures the relatedness or co-occurrence of two words. For example, 'coast' and 'shore' are very similar as they appear in the same context. At the same time 'clothes' and 'closet' are less similar because they are related but not interchangeable.

In [16]:
model.evaluate_word_pairs(test_data_dir + 'wordsim353.tsv')

/Users/akshay/Desktop/Incub/env/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `evaluate_word_pairs` (Method will be removed in 4.0.0, use self.wv.evaluate_word_pairs() instead).
  """Entry point for launching an IPython kernel.
2017-12-10 22:48:16,104 : INFO : Pearson correlation coefficient against /Users/akshay/Desktop/Incub/env/lib/python2.7/site-packages/gensim/test/test_data/wordsim353.tsv: 0.0869
2017-12-10 22:48:16,106 : INFO : Spearman rank-order correlation coefficient against /Users/akshay/Desktop/Incub/env/lib/python2.7/site-packages/gensim/test/test_data/wordsim353.tsv: 0.0830
2017-12-10 22:48:16,107 : INFO : Pairs with unknown words ratio: 85.6%


((0.086931919367035815, 0.54413143273212494),
 SpearmanrResult(correlation=0.082950560209212543, pvalue=0.56279273291255216),
 85.55240793201133)

Once again, **good performance on Google's or WS-353 test set doesn’t mean word2vec will work well in your application, or vice versa**. It’s always best to evaluate directly on your intended task. For an example of how to use word2vec in a classifier pipeline, see this [tutorial](https://github.com/RaRe-Technologies/movie-plots-by-genre).

## Storing and loading models
You can store/load models using the standard gensim methods:

In [17]:
from tempfile import mkstemp

fs, temp_path = mkstemp("gensim_temp")  # creates a temp file

model.save(temp_path)  # save the model

2017-12-10 22:48:16,130 : INFO : saving Word2Vec object under /var/folders/lf/jbtx3q5d7nv0xmfmz9r5d9nr0000gn/T/tmpg_WkKlgensim_temp, separately None
2017-12-10 22:48:16,132 : INFO : not storing attribute syn0norm
2017-12-10 22:48:16,134 : INFO : not storing attribute cum_table
2017-12-10 22:48:16,164 : INFO : saved /var/folders/lf/jbtx3q5d7nv0xmfmz9r5d9nr0000gn/T/tmpg_WkKlgensim_temp


In [18]:
new_model = gensim.models.Word2Vec.load(temp_path)  # open the model

2017-12-10 22:48:16,177 : INFO : loading Word2Vec object from /var/folders/lf/jbtx3q5d7nv0xmfmz9r5d9nr0000gn/T/tmpg_WkKlgensim_temp
2017-12-10 22:48:16,192 : INFO : loading wv recursively from /var/folders/lf/jbtx3q5d7nv0xmfmz9r5d9nr0000gn/T/tmpg_WkKlgensim_temp.wv.* with mmap=None
2017-12-10 22:48:16,194 : INFO : setting ignored attribute syn0norm to None
2017-12-10 22:48:16,195 : INFO : setting ignored attribute cum_table to None
2017-12-10 22:48:16,197 : INFO : loaded /var/folders/lf/jbtx3q5d7nv0xmfmz9r5d9nr0000gn/T/tmpg_WkKlgensim_temp


which uses pickle internally, optionally `mmap`‘ing the model’s internal large NumPy matrices into virtual memory directly from disk files, for inter-process memory sharing.

In addition, you can load models created by the original C tool, both using its text and binary formats:
```
  model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.txt', binary=False)
  # using gzipped/bz2 input works too, no need to unzip:
  model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.bin.gz', binary=True)
```

## Online training / Resuming training
Advanced users can load a model and continue training it with more sentences and [new vocabulary words](online_w2v_tutorial.ipynb):

In [19]:
model = gensim.models.Word2Vec.load(temp_path)
more_sentences = [['Advanced', 'users', 'can', 'load', 'a', 'model', 'and', 'continue', 'training', 'it', 'with', 'more', 'sentences']]
model.build_vocab(more_sentences, update=True)
model.train(more_sentences, total_examples=model.corpus_count, epochs=model.iter)

# cleaning up temp
os.close(fs)
os.remove(temp_path)

2017-12-10 22:48:16,222 : INFO : loading Word2Vec object from /var/folders/lf/jbtx3q5d7nv0xmfmz9r5d9nr0000gn/T/tmpg_WkKlgensim_temp
2017-12-10 22:48:16,236 : INFO : loading wv recursively from /var/folders/lf/jbtx3q5d7nv0xmfmz9r5d9nr0000gn/T/tmpg_WkKlgensim_temp.wv.* with mmap=None
2017-12-10 22:48:16,238 : INFO : setting ignored attribute syn0norm to None
2017-12-10 22:48:16,240 : INFO : setting ignored attribute cum_table to None
2017-12-10 22:48:16,241 : INFO : loaded /var/folders/lf/jbtx3q5d7nv0xmfmz9r5d9nr0000gn/T/tmpg_WkKlgensim_temp
2017-12-10 22:48:16,249 : INFO : collecting all words and their counts
2017-12-10 22:48:16,250 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-12-10 22:48:16,252 : INFO : collected 13 word types from a corpus of 13 raw words and 1 sentences
2017-12-10 22:48:16,253 : INFO : Updating model with new vocabulary
2017-12-10 22:48:16,255 : INFO : New added 0 unique words (0% of original 13) and increased the count of 0 pre-ex

You may need to tweak the `total_words` parameter to `train()`, depending on what learning rate decay you want to simulate.

Note that it’s not possible to resume training with models generated by the C tool, `KeyedVectors.load_word2vec_format()`. You can still use them for querying/similarity, but information vital for training (the vocab tree) is missing there.

## Using the model
`Word2Vec` supports several word similarity tasks out of the box:

In [20]:
model.most_similar(positive=['human', 'crime'], negative=['party'], topn=1)

/Users/akshay/Desktop/Incub/env/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
2017-12-10 22:48:16,307 : INFO : precomputing L2-norms of word weight vectors


[('longer', 0.987393856048584)]

In [21]:
model.doesnt_match("input is lunch he sentence cat".split())

/Users/akshay/Desktop/Incub/env/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
2017-12-10 22:48:16,329 : WARNING : vectors for words set(['lunch', 'input', 'cat']) are not present in the model, ignoring these words


'sentence'

In [22]:
print(model.similarity('human', 'party'))
print(model.similarity('tree', 'murder'))

0.999183292361
0.995676652317


/Users/akshay/Desktop/Incub/env/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.
/Users/akshay/Desktop/Incub/env/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


You can get the probability distribution for the center word given the context words as input:

In [23]:
print(model.predict_output_word(['emergency', 'beacon', 'received']))

[('more', 0.0010525994), ('continue', 0.00091269269), ('can', 0.00090120145), ('training', 0.00088642573), ('it', 0.00078034378), ('australia', 0.00075161911), ('there', 0.00074340496), ('government', 0.00074115628), ('or', 0.00073922565), ('could', 0.00073906122)]


The results here don't look good because the training corpus is very small. To get meaningful results one needs to train on 500k+ words.

If you need the raw output vectors in your application, you can access these either on a word-by-word basis:

In [24]:
model['tree']  # raw NumPy vector of a word

/Users/akshay/Desktop/Incub/env/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.0047541 ,  0.02339038, -0.03008565, -0.00900371, -0.03345428,
       -0.03905049,  0.00764565, -0.09334223, -0.01347537, -0.02327306,
        0.02428858, -0.02501613, -0.06924772, -0.0206782 , -0.02230095,
        0.04937958, -0.02983231, -0.00235232,  0.02352054, -0.02795854,
       -0.06937864, -0.00583119,  0.07089194,  0.05202649,  0.02536332,
        0.01430937, -0.00439524,  0.01913494, -0.01512169,  0.00164882,
        0.00091479,  0.07461091,  0.00978884, -0.00262773,  0.02442353,
        0.02157784, -0.04492744,  0.01839372,  0.01287366, -0.07190976,
       -0.08060756,  0.00635041, -0.04619569, -0.02844981,  0.03189611,
       -0.05828807,  0.03985275,  0.00480215, -0.01349906,  0.04141076,
        0.01602032,  0.00686777, -0.03806682,  0.00653816,  0.01565201,
       -0.01968512,  0.059731  , -0.0305224 ,  0.02148207, -0.0365483 ,
       -0.03145985, -0.05202309,  0.01823615, -0.03333651, -0.01165513,
       -0.06709775, -0.08279951,  0.0398951 ,  0.0558244 , -0.00

…or en-masse as a 2D NumPy matrix from `model.wv.syn0`.

## Training Loss Computation

The parameter `compute_loss` can be used to toggle computation of loss while training the Word2Vec model. The computed loss is stored in the model attribute `running_training_loss` and can be retrieved using the function `get_latest_training_loss` as follows : 

In [25]:
# instantiating and training the Word2Vec model
model_with_loss = gensim.models.Word2Vec(sentences, min_count=1, compute_loss=True, hs=0, sg=1, seed=42)

# getting the training loss value
training_loss = model_with_loss.get_latest_training_loss()
print(training_loss)

2017-12-10 22:48:16,406 : INFO : collecting all words and their counts
2017-12-10 22:48:16,410 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-12-10 22:48:16,447 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2017-12-10 22:48:16,448 : INFO : Loading a fresh vocabulary
2017-12-10 22:48:16,527 : INFO : min_count=1 retains 10186 unique words (100% of original 10186, drops 0)
2017-12-10 22:48:16,529 : INFO : min_count=1 leaves 59890 word corpus (100% of original 59890, drops 0)
2017-12-10 22:48:16,585 : INFO : deleting the raw counts dictionary of 10186 items
2017-12-10 22:48:16,586 : INFO : sample=0.001 downsamples 37 most-common words
2017-12-10 22:48:16,588 : INFO : downsampling leaves estimated 47231 word corpus (78.9% of prior 59890)
2017-12-10 22:48:16,590 : INFO : estimated required memory for 10186 words and 100 dimensions: 13241800 bytes
2017-12-10 22:48:16,638 : INFO : resetting layer weights
2017-12-10 22:48

1491250.25


### Benchmarks to see effect of training loss compuation code on training time

We first download and setup the test data used for getting the benchmarks.

In [26]:
input_data_files = []

def setup_input_data():
    # check if test data already present
    if os.path.isfile('./text8') is False:

        # download and decompress 'text8' corpus
        import zipfile
        ! wget 'http://mattmahoney.net/dc/text8.zip'
        ! unzip 'text8.zip'
    
        # create 1 MB, 10 MB and 50 MB files
        ! head -c1000000 text8 > text8_1000000
        ! head -c10000000 text8 > text8_10000000
        ! head -c50000000 text8 > text8_50000000
                
    # add 25 KB test file
    input_data_files.append(os.path.join(os.getcwd(), '../../gensim/test/test_data/lee_background.cor'))

    # add 1 MB test file
    input_data_files.append(os.path.join(os.getcwd(), 'text8_1000000'))

    # add 10 MB test file
    input_data_files.append(os.path.join(os.getcwd(), 'text8_10000000'))

    # add 50 MB test file
    input_data_files.append(os.path.join(os.getcwd(), 'text8_50000000'))

    # add 100 MB test file
    input_data_files.append(os.path.join(os.getcwd(), 'text8'))

setup_input_data()
print(input_data_files)

/bin/sh: wget: command not found
unzip:  cannot find or open text8.zip, text8.zip.zip or text8.zip.ZIP.
head: text8: No such file or directory
head: text8: No such file or directory
head: text8: No such file or directory
['/Users/akshay/Desktop/Incub/gensim/docs/notebooks/../../gensim/test/test_data/lee_background.cor', '/Users/akshay/Desktop/Incub/gensim/docs/notebooks/text8_1000000', '/Users/akshay/Desktop/Incub/gensim/docs/notebooks/text8_10000000', '/Users/akshay/Desktop/Incub/gensim/docs/notebooks/text8_50000000', '/Users/akshay/Desktop/Incub/gensim/docs/notebooks/text8']


We now compare the training time taken for different combinations of input data and model training parameters like `hs` and `sg`.

In [30]:
# using 25 KB and 50 MB files only for generating o/p -> comment next line for using all 5 test files
input_data_files = [input_data_files[0], input_data_files[-2]]
print(input_data_files)

import time
import numpy as np
import pandas as pd

train_time_values = []
seed_val = 42
sg_values = [0, 1]
hs_values = [0, 1]

for data_file in input_data_files:
    data = gensim.models.word2vec.LineSentence(data_file) 
    for sg_val in sg_values:
        for hs_val in hs_values:
            for loss_flag in [True, False]:
                time_taken_list = []
                for i in range(3):
                    start_time = time.time()
                    w2v_model = gensim.models.Word2Vec(data, compute_loss=loss_flag, sg=sg_val, hs=hs_val, seed=seed_val) 
                    time_taken_list.append(time.time() - start_time)

                time_taken_list = np.array(time_taken_list)
                time_mean = np.mean(time_taken_list)
                time_std = np.std(time_taken_list)
                train_time_values.append({'train_data': data_file, 'compute_loss': loss_flag, 'sg': sg_val, 'hs': hs_val, 'mean': time_mean, 'std': time_std})

train_times_table = pd.DataFrame(train_time_values)
train_times_table = train_times_table.sort_values(by=['train_data', 'sg', 'hs', 'compute_loss'], ascending=[False, False, True, False])
print(train_times_table)

2017-12-10 22:49:44,183 : INFO : collecting all words and their counts
2017-12-10 22:49:44,185 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-12-10 22:49:44,227 : INFO : collected 10781 word types from a corpus of 59890 raw words and 300 sentences
2017-12-10 22:49:44,228 : INFO : Loading a fresh vocabulary
2017-12-10 22:49:44,245 : INFO : min_count=5 retains 1762 unique words (16% of original 10781, drops 9019)
2017-12-10 22:49:44,247 : INFO : min_count=5 leaves 46084 word corpus (76% of original 59890, drops 13806)
2017-12-10 22:49:44,266 : INFO : deleting the raw counts dictionary of 10781 items
2017-12-10 22:49:44,271 : INFO : sample=0.001 downsamples 45 most-common words
2017-12-10 22:49:44,273 : INFO : downsampling leaves estimated 32610 word corpus (70.8% of prior 46084)
2017-12-10 22:49:44,274 : INFO : estimated required memory for 1762 words and 100 dimensions: 2290600 bytes
2017-12-10 22:49:44,285 : INFO : resetting layer weights
2017-12-10 22:

['/Users/akshay/Desktop/Incub/gensim/docs/notebooks/../../gensim/test/test_data/lee_background.cor', '/Users/akshay/Desktop/Incub/gensim/docs/notebooks/../../gensim/test/test_data/lee_background.cor']


2017-12-10 22:49:44,534 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-12-10 22:49:44,535 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-12-10 22:49:44,539 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-12-10 22:49:44,540 : INFO : training on 299450 raw words (162889 effective words) took 0.2s, 771050 effective words/s
2017-12-10 22:49:44,545 : INFO : collecting all words and their counts
2017-12-10 22:49:44,547 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-12-10 22:49:44,580 : INFO : collected 10781 word types from a corpus of 59890 raw words and 300 sentences
2017-12-10 22:49:44,581 : INFO : Loading a fresh vocabulary
2017-12-10 22:49:44,597 : INFO : min_count=5 retains 1762 unique words (16% of original 10781, drops 9019)
2017-12-10 22:49:44,599 : INFO : min_count=5 leaves 46084 word corpus (76% of original 59890, drops 13806)
2017-12-10 22:49:44,610 : INFO : deleting the raw

2017-12-10 22:49:46,337 : INFO : collected 10781 word types from a corpus of 59890 raw words and 300 sentences
2017-12-10 22:49:46,339 : INFO : Loading a fresh vocabulary
2017-12-10 22:49:46,355 : INFO : min_count=5 retains 1762 unique words (16% of original 10781, drops 9019)
2017-12-10 22:49:46,357 : INFO : min_count=5 leaves 46084 word corpus (76% of original 59890, drops 13806)
2017-12-10 22:49:46,370 : INFO : deleting the raw counts dictionary of 10781 items
2017-12-10 22:49:46,372 : INFO : sample=0.001 downsamples 45 most-common words
2017-12-10 22:49:46,374 : INFO : downsampling leaves estimated 32610 word corpus (70.8% of prior 46084)
2017-12-10 22:49:46,376 : INFO : estimated required memory for 1762 words and 100 dimensions: 3347800 bytes
2017-12-10 22:49:46,382 : INFO : constructing a huffman tree from 1762 words
2017-12-10 22:49:46,457 : INFO : built huffman tree with maximum node depth 13
2017-12-10 22:49:46,464 : INFO : resetting layer weights
2017-12-10 22:49:46,504 : IN

2017-12-10 22:49:49,217 : INFO : collecting all words and their counts
2017-12-10 22:49:49,219 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-12-10 22:49:49,260 : INFO : collected 10781 word types from a corpus of 59890 raw words and 300 sentences
2017-12-10 22:49:49,263 : INFO : Loading a fresh vocabulary
2017-12-10 22:49:49,279 : INFO : min_count=5 retains 1762 unique words (16% of original 10781, drops 9019)
2017-12-10 22:49:49,280 : INFO : min_count=5 leaves 46084 word corpus (76% of original 59890, drops 13806)
2017-12-10 22:49:49,295 : INFO : deleting the raw counts dictionary of 10781 items
2017-12-10 22:49:49,297 : INFO : sample=0.001 downsamples 45 most-common words
2017-12-10 22:49:49,299 : INFO : downsampling leaves estimated 32610 word corpus (70.8% of prior 46084)
2017-12-10 22:49:49,300 : INFO : estimated required memory for 1762 words and 100 dimensions: 3347800 bytes
2017-12-10 22:49:49,309 : INFO : constructing a huffman tree from 1762 

...

2017-12-10 22:50:05,219 : INFO : deleting the raw counts dictionary of 10781 items
2017-12-10 22:50:05,222 : INFO : sample=0.001 downsamples 45 most-common words
2017-12-10 22:50:05,224 : INFO : downsampling leaves estimated 32610 word corpus (70.8% of prior 46084)
2017-12-10 22:50:05,226 : INFO : estimated required memory for 1762 words and 100 dimensions: 3347800 bytes
2017-12-10 22:50:05,230 : INFO : constructing a huffman tree from 1762 words
2017-12-10 22:50:05,303 : INFO : built huffman tree with maximum node depth 13
2017-12-10 22:50:05,310 : INFO : resetting layer weights
2017-12-10 22:50:05,353 : INFO : training model with 3 workers on 1762 vocabulary and 100 features, using sg=0 hs=1 sample=0.001 negative=5 window=5
2017-12-10 22:50:05,725 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-12-10 22:50:05,727 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-12-10 22:50:05,739 : INFO : worker thread finished; awaiting finish of 0 more

2017-12-10 22:50:08,104 : INFO : min_count=5 retains 1762 unique words (16% of original 10781, drops 9019)
2017-12-10 22:50:08,106 : INFO : min_count=5 leaves 46084 word corpus (76% of original 59890, drops 13806)
2017-12-10 22:50:08,119 : INFO : deleting the raw counts dictionary of 10781 items
2017-12-10 22:50:08,122 : INFO : sample=0.001 downsamples 45 most-common words
2017-12-10 22:50:08,123 : INFO : downsampling leaves estimated 32610 word corpus (70.8% of prior 46084)
2017-12-10 22:50:08,125 : INFO : estimated required memory for 1762 words and 100 dimensions: 2290600 bytes
2017-12-10 22:50:08,135 : INFO : resetting layer weights
2017-12-10 22:50:08,174 : INFO : training model with 3 workers on 1762 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5 window=5
2017-12-10 22:50:08,674 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-12-10 22:50:08,677 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-12-10 22:50:08,684 

2017-12-10 22:50:11,495 : INFO : resetting layer weights
2017-12-10 22:50:11,544 : INFO : training model with 3 workers on 1762 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5 window=5
2017-12-10 22:50:12,053 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-12-10 22:50:12,058 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-12-10 22:50:12,079 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-12-10 22:50:12,081 : INFO : training on 299450 raw words (162948 effective words) took 0.5s, 305208 effective words/s
2017-12-10 22:50:12,084 : INFO : collecting all words and their counts
2017-12-10 22:50:12,086 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-12-10 22:50:12,126 : INFO : collected 10781 word types from a corpus of 59890 raw words and 300 sentences
2017-12-10 22:50:12,127 : INFO : Loading a fresh vocabulary
2017-12-10 22:50:12,143 : INFO : min_count=5 retains 1762

2017-12-10 22:50:18,409 : INFO : min_count=5 leaves 46084 word corpus (76% of original 59890, drops 13806)
2017-12-10 22:50:18,424 : INFO : deleting the raw counts dictionary of 10781 items
2017-12-10 22:50:18,426 : INFO : sample=0.001 downsamples 45 most-common words
2017-12-10 22:50:18,427 : INFO : downsampling leaves estimated 32610 word corpus (70.8% of prior 46084)
2017-12-10 22:50:18,429 : INFO : estimated required memory for 1762 words and 100 dimensions: 3347800 bytes
2017-12-10 22:50:18,436 : INFO : constructing a huffman tree from 1762 words
2017-12-10 22:50:18,526 : INFO : built huffman tree with maximum node depth 13
2017-12-10 22:50:18,533 : INFO : resetting layer weights
2017-12-10 22:50:18,576 : INFO : training model with 3 workers on 1762 vocabulary and 100 features, using sg=1 hs=1 sample=0.001 negative=5 window=5
2017-12-10 22:50:19,580 : INFO : PROGRESS: at 76.13% examples, 123650 words/s, in_qsize 5, out_qsize 0
2017-12-10 22:50:19,796 : INFO : worker thread finishe

    compute_loss  hs      mean  sg       std  \
4           True   0  0.686046   1  0.034859   
12          True   0  0.669860   1  0.024516   
5          False   0  0.650021   1  0.010763   
13         False   0  0.675481   1  0.022741   
6           True   1  1.524250   1  0.101517   
14          True   1  1.559246   1  0.138129   
7          False   1  1.344446   1  0.015572   
15         False   1  1.549747   1  0.046282   
0           True   0  0.357842   0  0.003186   
8           True   0  0.349151   0  0.016225   
1          False   0  0.347047   0  0.004262   
9          False   0  0.361551   0  0.006024   
2           True   1  0.604596   0  0.048566   
10          True   1  0.584270   0  0.032410   
3          False   1  0.583010   0  0.044088   
11         False   1  0.561301   0  0.023942   

                                           train_data  
4   /Users/akshay/Desktop/Incub/gensim/docs/notebo...  
12  /Users/akshay/Desktop/Incub/gensim/docs/notebo...  
5   /Users/aksh

### Adding Word2Vec "model to dict" method to production pipeline

In [31]:
def model_to_dict():
    # method to find and store all most similar words to all words in the trained vocabulary.
    most_similars_precalc = {word : model.wv.most_similar(word) for word in model.wv.index2word}
    return most_similars_precalc

In [32]:
model_to_dict()

{'four': [('over', 0.9999396800994873),
  ('after', 0.9999361634254456),
  ('local', 0.9999359846115112),
  ('on', 0.9999353289604187),
  ('world', 0.9999334812164307),
  ('last', 0.9999321103096008),
  ('into', 0.9999313354492188),
  ('from', 0.9999313354492188),
  ('a', 0.999931275844574),
  ('at', 0.9999309778213501)],
 'jihad': [('on', 0.9995610117912292),
  ('and', 0.9995589256286621),
  ('gaza', 0.9995578527450562),
  ('in', 0.999553918838501),
  ('palestinian', 0.9995510578155518),
  ('from', 0.9995386600494385),
  ('former', 0.9995379447937012),
  ('were', 0.9995355010032654),
  ('security', 0.9995349049568176),
  ('an', 0.9995259642601013)],
 'captain': [('are', 0.9998462796211243),
  ('any', 0.9998350143432617),
  ('out', 0.9998338222503662),
  ('over', 0.9998304843902588),
  ('now', 0.9998301267623901),
  ('is', 0.9998231530189514),
  ('have', 0.9998231530189514),
  ('including', 0.9998214244842529),
  ('up', 0.999820351600647),
  ('at', 0.9998188018798828)],
 'gillespie': [

## Conclusion

In this tutorial we learned how to train word2vec models on your custom data and also how to evaluate it. Hope that you too will find this popular tool useful in your Machine Learning tasks!

## Links


Full `word2vec` API docs [here](http://radimrehurek.com/gensim/models/word2vec.html); get [gensim](http://radimrehurek.com/gensim/) here. Original C toolkit and `word2vec` papers by Google [here](https://code.google.com/archive/p/word2vec/).